In [13]:
# Dependencies to Visualize the model
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Filepaths, numpy, and Tensorflow
import os
import numpy as np
import keras

# Preprocessing imports
import glob
import os.path as path
import imageio
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# Model Imports
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, TensorBoard
from sklearn.metrics import accuracy_score, f1_score

from datetime import datetime

import cv2
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 

In [2]:
root = 'Data'

# Load the images
file_paths = []
labels = []
targets = []

emotions = ['angry','disgust','fear','happy','neutral','sad','surprise']
x=0
e=0
for emotion in emotions:
    print(emotion)
    for filename in glob.glob(f"{root}/{emotion}/*.jpg"):
        path = filename.split('\\')[0]+'/'+filename.split("\\")[1]
        file_paths.append(path)
        labels.append(e)
        targets.append(emotion)
        x+=1
    for filename in glob.glob(f"{root}/{emotion}/*.png"):
        path = filename.split('\\')[0]+'/'+filename.split("\\")[1]
        file_paths.append(path)
        labels.append(e)
        targets.append(emotion)
        x+=1
    e+=1
    x=0

angry
disgust
fear
happy
neutral
sad
surprise


In [3]:
# Load the images
images = []
image_size = (50,50)

for path in file_paths:
    print(f"{x} of {len(file_paths)}")
    x+=1
    img = load_img(path,target_size = image_size,color_mode='grayscale')
    img = img_to_array(img)
    images.append(img)

0 of 91652
1 of 91652
2 of 91652
3 of 91652
4 of 91652
5 of 91652
6 of 91652
7 of 91652
8 of 91652
9 of 91652
10 of 91652
11 of 91652
12 of 91652
13 of 91652
14 of 91652
15 of 91652
16 of 91652
17 of 91652
18 of 91652
19 of 91652
20 of 91652
21 of 91652
22 of 91652
23 of 91652
24 of 91652
25 of 91652
26 of 91652
27 of 91652
28 of 91652
29 of 91652
30 of 91652
31 of 91652
32 of 91652
33 of 91652
34 of 91652
35 of 91652
36 of 91652
37 of 91652
38 of 91652
39 of 91652
40 of 91652
41 of 91652
42 of 91652
43 of 91652
44 of 91652
45 of 91652
46 of 91652
47 of 91652
48 of 91652
49 of 91652
50 of 91652
51 of 91652
52 of 91652
53 of 91652
54 of 91652
55 of 91652
56 of 91652
57 of 91652
58 of 91652
59 of 91652
60 of 91652
61 of 91652
62 of 91652
63 of 91652
64 of 91652
65 of 91652
66 of 91652
67 of 91652
68 of 91652
69 of 91652
70 of 91652
71 of 91652
72 of 91652
73 of 91652
74 of 91652
75 of 91652
76 of 91652
77 of 91652
78 of 91652
79 of 91652
80 of 91652
81 of 91652
82 of 91652
83 of 91652
84

651 of 91652
652 of 91652
653 of 91652
654 of 91652
655 of 91652
656 of 91652
657 of 91652
658 of 91652
659 of 91652
660 of 91652
661 of 91652
662 of 91652
663 of 91652
664 of 91652
665 of 91652
666 of 91652
667 of 91652
668 of 91652
669 of 91652
670 of 91652
671 of 91652
672 of 91652
673 of 91652
674 of 91652
675 of 91652
676 of 91652
677 of 91652
678 of 91652
679 of 91652
680 of 91652
681 of 91652
682 of 91652
683 of 91652
684 of 91652
685 of 91652
686 of 91652
687 of 91652
688 of 91652
689 of 91652
690 of 91652
691 of 91652
692 of 91652
693 of 91652
694 of 91652
695 of 91652
696 of 91652
697 of 91652
698 of 91652
699 of 91652
700 of 91652
701 of 91652
702 of 91652
703 of 91652
704 of 91652
705 of 91652
706 of 91652
707 of 91652
708 of 91652
709 of 91652
710 of 91652
711 of 91652
712 of 91652
713 of 91652
714 of 91652
715 of 91652
716 of 91652
717 of 91652
718 of 91652
719 of 91652
720 of 91652
721 of 91652
722 of 91652
723 of 91652
724 of 91652
725 of 91652
726 of 91652
727 of 91652

1262 of 91652
1263 of 91652
1264 of 91652
1265 of 91652
1266 of 91652
1267 of 91652
1268 of 91652
1269 of 91652
1270 of 91652
1271 of 91652
1272 of 91652
1273 of 91652
1274 of 91652
1275 of 91652
1276 of 91652
1277 of 91652
1278 of 91652
1279 of 91652
1280 of 91652
1281 of 91652
1282 of 91652
1283 of 91652
1284 of 91652
1285 of 91652
1286 of 91652
1287 of 91652
1288 of 91652
1289 of 91652
1290 of 91652
1291 of 91652
1292 of 91652
1293 of 91652
1294 of 91652
1295 of 91652
1296 of 91652
1297 of 91652
1298 of 91652
1299 of 91652
1300 of 91652
1301 of 91652
1302 of 91652
1303 of 91652
1304 of 91652
1305 of 91652
1306 of 91652
1307 of 91652
1308 of 91652
1309 of 91652
1310 of 91652
1311 of 91652
1312 of 91652
1313 of 91652
1314 of 91652
1315 of 91652
1316 of 91652
1317 of 91652
1318 of 91652
1319 of 91652
1320 of 91652
1321 of 91652
1322 of 91652
1323 of 91652
1324 of 91652
1325 of 91652
1326 of 91652
1327 of 91652
1328 of 91652
1329 of 91652
1330 of 91652
1331 of 91652
1332 of 91652
1333 o

1853 of 91652
1854 of 91652
1855 of 91652
1856 of 91652
1857 of 91652
1858 of 91652
1859 of 91652
1860 of 91652
1861 of 91652
1862 of 91652
1863 of 91652
1864 of 91652
1865 of 91652
1866 of 91652
1867 of 91652
1868 of 91652
1869 of 91652
1870 of 91652
1871 of 91652
1872 of 91652
1873 of 91652
1874 of 91652
1875 of 91652
1876 of 91652
1877 of 91652
1878 of 91652
1879 of 91652
1880 of 91652
1881 of 91652
1882 of 91652
1883 of 91652
1884 of 91652
1885 of 91652
1886 of 91652
1887 of 91652
1888 of 91652
1889 of 91652
1890 of 91652
1891 of 91652
1892 of 91652
1893 of 91652
1894 of 91652
1895 of 91652
1896 of 91652
1897 of 91652
1898 of 91652
1899 of 91652
1900 of 91652
1901 of 91652
1902 of 91652
1903 of 91652
1904 of 91652
1905 of 91652
1906 of 91652
1907 of 91652
1908 of 91652
1909 of 91652
1910 of 91652
1911 of 91652
1912 of 91652
1913 of 91652
1914 of 91652
1915 of 91652
1916 of 91652
1917 of 91652
1918 of 91652
1919 of 91652
1920 of 91652
1921 of 91652
1922 of 91652
1923 of 91652
1924 o

2441 of 91652
2442 of 91652
2443 of 91652
2444 of 91652
2445 of 91652
2446 of 91652
2447 of 91652
2448 of 91652
2449 of 91652
2450 of 91652
2451 of 91652
2452 of 91652
2453 of 91652
2454 of 91652
2455 of 91652
2456 of 91652
2457 of 91652
2458 of 91652
2459 of 91652
2460 of 91652
2461 of 91652
2462 of 91652
2463 of 91652
2464 of 91652
2465 of 91652
2466 of 91652
2467 of 91652
2468 of 91652
2469 of 91652
2470 of 91652
2471 of 91652
2472 of 91652
2473 of 91652
2474 of 91652
2475 of 91652
2476 of 91652
2477 of 91652
2478 of 91652
2479 of 91652
2480 of 91652
2481 of 91652
2482 of 91652
2483 of 91652
2484 of 91652
2485 of 91652
2486 of 91652
2487 of 91652
2488 of 91652
2489 of 91652
2490 of 91652
2491 of 91652
2492 of 91652
2493 of 91652
2494 of 91652
2495 of 91652
2496 of 91652
2497 of 91652
2498 of 91652
2499 of 91652
2500 of 91652
2501 of 91652
2502 of 91652
2503 of 91652
2504 of 91652
2505 of 91652
2506 of 91652
2507 of 91652
2508 of 91652
2509 of 91652
2510 of 91652
2511 of 91652
2512 o

...

89247 of 91652
89248 of 91652
89249 of 91652
89250 of 91652
89251 of 91652
89252 of 91652
89253 of 91652
89254 of 91652
89255 of 91652
89256 of 91652
89257 of 91652
89258 of 91652
89259 of 91652
89260 of 91652
89261 of 91652
89262 of 91652
89263 of 91652
89264 of 91652
89265 of 91652
89266 of 91652
89267 of 91652
89268 of 91652
89269 of 91652
89270 of 91652
89271 of 91652
89272 of 91652
89273 of 91652
89274 of 91652
89275 of 91652
89276 of 91652
89277 of 91652
89278 of 91652
89279 of 91652
89280 of 91652
89281 of 91652
89282 of 91652
89283 of 91652
89284 of 91652
89285 of 91652
89286 of 91652
89287 of 91652
89288 of 91652
89289 of 91652
89290 of 91652
89291 of 91652
89292 of 91652
89293 of 91652
89294 of 91652
89295 of 91652
89296 of 91652
89297 of 91652
89298 of 91652
89299 of 91652
89300 of 91652
89301 of 91652
89302 of 91652
89303 of 91652
89304 of 91652
89305 of 91652
89306 of 91652
89307 of 91652
89308 of 91652
89309 of 91652
89310 of 91652
89311 of 91652
89312 of 91652
89313 of 9

89801 of 91652
89802 of 91652
89803 of 91652
89804 of 91652
89805 of 91652
89806 of 91652
89807 of 91652
89808 of 91652
89809 of 91652
89810 of 91652
89811 of 91652
89812 of 91652
89813 of 91652
89814 of 91652
89815 of 91652
89816 of 91652
89817 of 91652
89818 of 91652
89819 of 91652
89820 of 91652
89821 of 91652
89822 of 91652
89823 of 91652
89824 of 91652
89825 of 91652
89826 of 91652
89827 of 91652
89828 of 91652
89829 of 91652
89830 of 91652
89831 of 91652
89832 of 91652
89833 of 91652
89834 of 91652
89835 of 91652
89836 of 91652
89837 of 91652
89838 of 91652
89839 of 91652
89840 of 91652
89841 of 91652
89842 of 91652
89843 of 91652
89844 of 91652
89845 of 91652
89846 of 91652
89847 of 91652
89848 of 91652
89849 of 91652
89850 of 91652
89851 of 91652
89852 of 91652
89853 of 91652
89854 of 91652
89855 of 91652
89856 of 91652
89857 of 91652
89858 of 91652
89859 of 91652
89860 of 91652
89861 of 91652
89862 of 91652
89863 of 91652
89864 of 91652
89865 of 91652
89866 of 91652
89867 of 9

90351 of 91652
90352 of 91652
90353 of 91652
90354 of 91652
90355 of 91652
90356 of 91652
90357 of 91652
90358 of 91652
90359 of 91652
90360 of 91652
90361 of 91652
90362 of 91652
90363 of 91652
90364 of 91652
90365 of 91652
90366 of 91652
90367 of 91652
90368 of 91652
90369 of 91652
90370 of 91652
90371 of 91652
90372 of 91652
90373 of 91652
90374 of 91652
90375 of 91652
90376 of 91652
90377 of 91652
90378 of 91652
90379 of 91652
90380 of 91652
90381 of 91652
90382 of 91652
90383 of 91652
90384 of 91652
90385 of 91652
90386 of 91652
90387 of 91652
90388 of 91652
90389 of 91652
90390 of 91652
90391 of 91652
90392 of 91652
90393 of 91652
90394 of 91652
90395 of 91652
90396 of 91652
90397 of 91652
90398 of 91652
90399 of 91652
90400 of 91652
90401 of 91652
90402 of 91652
90403 of 91652
90404 of 91652
90405 of 91652
90406 of 91652
90407 of 91652
90408 of 91652
90409 of 91652
90410 of 91652
90411 of 91652
90412 of 91652
90413 of 91652
90414 of 91652
90415 of 91652
90416 of 91652
90417 of 9

90899 of 91652
90900 of 91652
90901 of 91652
90902 of 91652
90903 of 91652
90904 of 91652
90905 of 91652
90906 of 91652
90907 of 91652
90908 of 91652
90909 of 91652
90910 of 91652
90911 of 91652
90912 of 91652
90913 of 91652
90914 of 91652
90915 of 91652
90916 of 91652
90917 of 91652
90918 of 91652
90919 of 91652
90920 of 91652
90921 of 91652
90922 of 91652
90923 of 91652
90924 of 91652
90925 of 91652
90926 of 91652
90927 of 91652
90928 of 91652
90929 of 91652
90930 of 91652
90931 of 91652
90932 of 91652
90933 of 91652
90934 of 91652
90935 of 91652
90936 of 91652
90937 of 91652
90938 of 91652
90939 of 91652
90940 of 91652
90941 of 91652
90942 of 91652
90943 of 91652
90944 of 91652
90945 of 91652
90946 of 91652
90947 of 91652
90948 of 91652
90949 of 91652
90950 of 91652
90951 of 91652
90952 of 91652
90953 of 91652
90954 of 91652
90955 of 91652
90956 of 91652
90957 of 91652
90958 of 91652
90959 of 91652
90960 of 91652
90961 of 91652
90962 of 91652
90963 of 91652
90964 of 91652
90965 of 9

91452 of 91652
91453 of 91652
91454 of 91652
91455 of 91652
91456 of 91652
91457 of 91652
91458 of 91652
91459 of 91652
91460 of 91652
91461 of 91652
91462 of 91652
91463 of 91652
91464 of 91652
91465 of 91652
91466 of 91652
91467 of 91652
91468 of 91652
91469 of 91652
91470 of 91652
91471 of 91652
91472 of 91652
91473 of 91652
91474 of 91652
91475 of 91652
91476 of 91652
91477 of 91652
91478 of 91652
91479 of 91652
91480 of 91652
91481 of 91652
91482 of 91652
91483 of 91652
91484 of 91652
91485 of 91652
91486 of 91652
91487 of 91652
91488 of 91652
91489 of 91652
91490 of 91652
91491 of 91652
91492 of 91652
91493 of 91652
91494 of 91652
91495 of 91652
91496 of 91652
91497 of 91652
91498 of 91652
91499 of 91652
91500 of 91652
91501 of 91652
91502 of 91652
91503 of 91652
91504 of 91652
91505 of 91652
91506 of 91652
91507 of 91652
91508 of 91652
91509 of 91652
91510 of 91652
91511 of 91652
91512 of 91652
91513 of 91652
91514 of 91652
91515 of 91652
91516 of 91652
91517 of 91652
91518 of 9

In [4]:
images = np.asarray(images)       
len(images)  

91652

In [5]:
images[0]

array([[[ 72.],
        [ 78.],
        [ 81.],
        ...,
        [ 52.],
        [ 43.],
        [ 40.]],

       [[ 65.],
        [ 62.],
        [ 59.],
        ...,
        [ 55.],
        [ 52.],
        [ 45.]],

       [[ 47.],
        [ 42.],
        [ 60.],
        ...,
        [ 47.],
        [ 54.],
        [ 49.]],

       ...,

       [[ 91.],
        [ 65.],
        [ 46.],
        ...,
        [ 74.],
        [ 54.],
        [ 43.]],

       [[ 76.],
        [ 80.],
        [ 76.],
        ...,
        [108.],
        [ 69.],
        [ 46.]],

       [[ 77.],
        [ 74.],
        [ 81.],
        ...,
        [105.],
        [108.],
        [ 82.]]], dtype=float32)

In [6]:
# Get image size
image_size = np.asarray([images.shape[1], images.shape[2], images.shape[3]])
print(image_size)

[50 50  1]


In [7]:
df = pd.DataFrame(list(zip(targets, labels,images,file_paths)),columns=["emotions","labels","images","paths"])
df.head()

,emotions,labels,images,paths
0,angry,0,"[[[72.0], [78.0], [81.0], [75.0], [59.0], [54....",Data/angry/0.jpg
1,angry,0,"[[[152.0], [149.0], [147.0], [157.0], [146.0],...",Data/angry/1.jpg
2,angry,0,"[[[29.0], [25.0], [21.0], [23.0], [26.0], [24....",Data/angry/10.jpg
3,angry,0,"[[[32.0], [23.0], [20.0], [56.0], [43.0], [34....",Data/angry/10002.jpg
4,angry,0,"[[[222.0], [218.0], [202.0], [189.0], [199.0],...",Data/angry/10016.jpg


In [8]:
def imageProcess(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x,y,w,h) in faces:
        face_clip = img[y:y+h, x:x+w]  #cropping the face in image
        img = cv2.resize(face_clip, (350, 350))  #resizing image
    return img 

In [17]:
processed = []

In [ ]:

for i, row in enumerate(df.itertuples()):
#     try:
    path = row.paths
    img = imageProcess(path)
    processed.append(img)
#     df.loc[i,"image_process"] = img
#     except:
#         print(path)
#         df.loc[i,"image_process"] = np.nan

# emotions.head()

In [ ]:
# Scale
images = images / 255


#Split into training and testing data
X = images
y = labels
from sklearn.model_selection import train_test_split
# 80/10/10 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.111, random_state=1)

X_train.shape

In [ ]:
X_val.shape

In [ ]:
X_test.shape

In [ ]:
labels

In [ ]:
from keras.utils import to_categorical
num_classes=7

# Prepare y data
y_binary_train = to_categorical(y_train, num_classes=num_classes, dtype='float32')
y_binary_test = to_categorical(y_test, num_classes=num_classes, dtype='float32')

In [ ]:
y_binary_test

In [ ]:
#designing the CNN
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# Print a summary of the model
model.summary()

In [ ]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=PATIENCE, verbose=2, mode='auto')

In [ ]:
# Place the callbacks in a list
callbacks = [early_stopping]

In [ ]:
# Train the model
model.fit(np.array(X_train), np.array(y_train), epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks, verbose=3)

In [ ]:
# Make a prediction on the test set
test_predictions = model.predict(X_test)
test_predictions = np.argmax(test_predictions , axis=1)
test_predictions

In [ ]:
# Report the accuracy
accuracy = accuracy_score(y_test, test_predictions)
print("Accuracy: " + str(accuracy))

In [ ]:
 # Save the model
# model.save("emotion_model_trained.h5")